# Anti-Collatz exploration (reproducible notebook)

This notebook reproduces the core figures in this repository and documents the **Anti-Collatz** map.

## Maps

**Classical Collatz**
- even: \(n\mapsto n/2\)
- odd:  \(n\mapsto 3n+1\)

**Anti-Collatz (parity-inverted)**
- even: \(n\mapsto 3n+1\)
- odd:  \(n\mapsto \lfloor n/2\rfloor\)

> Note on naming: “anti-Collatz” is used informally online for several related constructions.
> In this repo we reserve the term for the *parity-inverted* map above.


In [ ]:
import matplotlib.pyplot as plt
from pathlib import Path

def collatz_step(n: int) -> int:
    return n // 2 if n % 2 == 0 else 3*n + 1

def anti_collatz_step(n: int) -> int:
    return 3*n + 1 if n % 2 == 0 else n // 2

def iterate(f, start: int, steps: int = 60):
    seq = [start]
    for _ in range(steps):
        seq.append(f(seq[-1]))
    return seq

outdir = Path("../figures")
outdir.mkdir(exist_ok=True)


## Trajectories: Collatz vs Anti-Collatz

We plot a small set of example seeds under both maps. The same seeds look radically different
under the parity inversion.


In [ ]:
starts = [3, 5, 6, 7, 10, 12]
steps = 60


In [ ]:
# Classical Collatz trajectories
plt.figure(figsize=(8,6))
for s in starts:
    y = iterate(collatz_step, s, steps)
    plt.plot(range(len(y)), y, label=f"start={s}")
plt.xlabel("Step")
plt.ylabel("Value")
plt.title("Classical Collatz trajectories (example seeds)")
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.savefig(outdir / "collatz_trajectories.png", dpi=180)
plt.show()


In [ ]:
# Anti-Collatz trajectories
plt.figure(figsize=(8,6))
for s in starts:
    y = iterate(anti_collatz_step, s, steps)
    plt.plot(range(len(y)), y, label=f"start={s}")
plt.xlabel("Step")
plt.ylabel("Value")
plt.title("Anti-Collatz trajectories (example seeds)")
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.savefig(outdir / "anti_collatz_trajectories.png", dpi=180)
plt.show()


## Cycle detection (Anti-Collatz)

We detect cycles by recording visited states. When a value repeats, the orbit has entered a loop.

A simple non-trivial example is the 6-cycle:
\[
6 \to 19 \to 9 \to 4 \to 13 \to 6
\]


In [ ]:
def detect_cycle(f, seed: int, max_iter: int = 5000):
    seen = {}
    trace = []
    x = seed
    for _ in range(max_iter):
        if x in seen:
            start = seen[x]
            trace.append(x)
            return trace, start, len(trace)-1
        seen[x] = len(trace)
        trace.append(x)
        x = f(x)
    return trace, None, None

trace, cs, ce = detect_cycle(anti_collatz_step, 6, 2000)
cycle = trace[cs:ce+1] if cs is not None else None
cycle


In [ ]:
# Plot with cycle region highlighted
plt.figure(figsize=(8,6))
plt.plot(range(len(trace)), trace)
plt.xlabel("Step")
plt.ylabel("Value")
plt.title("Anti-Collatz sequence with cycle highlighted (start=6)")
plt.grid(True)
if cs is not None:
    plt.axvspan(cs, ce, alpha=0.3)
plt.tight_layout()
plt.savefig(outdir / "anti_collatz_cycle_start6.png", dpi=180)
plt.show()


## Notes on novelty / wording

This parity-inverted map is a natural member of generalized Collatz-type functions.
The main contribution of this repo is to **name and define it unambiguously**, and to provide
**reproducible empirical exploration** (plots + cycle detection) in one place.

For scientific clarity, the README and research note avoid claiming that the map is “unknown”;
instead they state that the **specific term** and **explicit framing** are not standardized.


## Overlay comparison: Collatz vs Anti-Collatz

Same seeds, same step count. The dashed curves are Anti-Collatz.

In [ ]:
# Overlay plot (dashed = Anti-Collatz)
plt.figure(figsize=(8,6))
for s in starts:
    y = iterate(collatz_step, s, steps)
    plt.plot(range(len(y)), y, label=f"Collatz start={s}")
for s in starts:
    y = iterate(anti_collatz_step, s, steps)
    plt.plot(range(len(y)), y, linestyle="--", label=f"Anti start={s}")
plt.xlabel("Step")
plt.ylabel("Value")
plt.title("Collatz vs Anti-Collatz (same seeds; dashed = Anti)")
plt.grid(True)
plt.legend(ncol=2, fontsize=8)
plt.tight_layout()
plt.savefig(outdir / "collatz_vs_anti_overlay.png", dpi=180)
plt.show()


## Small cycle census (Anti-Collatz)

We scan seeds 1..N, detect the first repeated state (within a finite iteration cap), canonicalize cycles
up to rotation, and save a small CSV dataset.

In [ ]:
import pandas as pd

def canonical_cycle(cycle):
    if cycle and cycle[0] == cycle[-1]:
        cycle = cycle[:-1]
    m = len(cycle)
    if m == 0:
        return tuple()
    rots = [tuple(cycle[i:]+cycle[:i]) for i in range(m)]
    return min(rots)

def anti_cycle_census(N=5000, max_iter=20000):
    cycle_map = {}
    for seed in range(1, N+1):
        trace, cs, ce = detect_cycle(anti_collatz_step, seed, max_iter=max_iter)
        if cs is None:
            continue
        cyc = trace[cs:ce+1]
        can = canonical_cycle(cyc)
        cycle_map.setdefault(can, []).append(seed)

    rows = []
    for cyc, seeds in cycle_map.items():
        rows.append({
            "cycle_length": len(cyc),
            "cycle": " -> ".join(map(str, cyc)) + f" -> {cyc[0]}",
            "min_element": min(cyc) if cyc else None,
            "seed_count_1_to_N": len(seeds),
            "example_seeds": ", ".join(map(str, seeds[:10])) + (" ..." if len(seeds) > 10 else "")
        })
    df = pd.DataFrame(rows).sort_values(["cycle_length","min_element"]).reset_index(drop=True)
    return df

df = anti_cycle_census(N=5000, max_iter=20000)
df.head(10)


In [ ]:
# Save dataset
csv_path = Path("../data/anti_cycles_up_to_5000.csv")
csv_path.parent.mkdir(exist_ok=True)
df.to_csv(csv_path, index=False)
csv_path


In [ ]:
# Cycle-length histogram (unique cycles)
plt.figure(figsize=(8,6))
if len(df):
    plt.hist(df["cycle_length"], bins=range(1, int(df["cycle_length"].max())+2))
plt.xlabel("Cycle length (unique cycles)")
plt.ylabel("Count")
plt.title("Anti-Collatz: distribution of detected cycle lengths (seeds 1..5000)")
plt.grid(True)
plt.tight_layout()
plt.savefig(outdir / "anti_cycle_length_hist_1_to_5000.png", dpi=180)
plt.show()
